In [ ]:
# Semantic Clustering Analysis for Language Model Outputs
# Date: May 15, 2025
# Description: This script performs a semantic analysis of language model outputs using bigram extraction,
# spectral clustering, t-SNE visualization, and Phi coefficient-based semantic labeling.
# It compares six models (AstroSage, AstroLlama2, Claude 3.7 Sonnet, Grok 3 Mini (high), Llama 3.1 8B, o4-mini)
# against the PSA OpenBench Gamma dataset, as described in the accompanying paper.

import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import SpectralClustering
from collections import Counter
from scipy.stats import chi2_contingency
import warnings

# Configure matplotlib to use serif font for better readability in plots
plt.rcParams['font.family'] = 'serif'

def calculate_phi_coefficient(bigrams, labels, document_blocks, n_clusters):
    """
    Calculate Phi coefficients to quantify the association between bigrams and semantic categories.

    Args:
        bigrams (list): List of bigrams extracted from model outputs.
        labels (np.ndarray): Cluster labels assigned to each bigram.
        document_blocks (list): List of text blocks (replies) from model outputs.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Phi scores for each cluster, mapping clusters to category scores (e.g., math, code).
    """
    # Initialize a dictionary to store Phi scores for each cluster and semantic category
    phi_scores = {i: Counter() for i in range(n_clusters)}
    doc_types = ['math', 'code', 'literary', 'dynamics']

    # Classify each document block into a semantic category using regex patterns
    doc_classifications = []
    for block in document_blocks:
        block_lower = block.lower()
        if re.search(r'[0-9]+(\.[0-9]+)?|[μΔ√πθ]', block_lower):  # Math: numbers and symbols
            doc_classifications.append('math')
        elif re.search(r'(def|class|if|for|while|int|str|return|[a-z_][a-z0-9_]*(\.[a-z0-9_]+)?)', block_lower):  # Code: keywords and identifiers
            doc_classifications.append('code')
        elif re.search(r'(process|change|interaction|system|flow|evolve|adapt|transition|state|time)', block_lower):  # Dynamics: processes and temporal terms
            doc_classifications.append('dynamics')
        else:  # Literary: default category for narrative or descriptive content
            doc_classifications.append('literary')

    total_docs = len(document_blocks)
    doc_type_counts = Counter(doc_classifications)

    # Compute Phi coefficient for each bigram and semantic category
    for i, bigram in enumerate(bigrams):
        cluster = labels[i]
        bigram_lower = bigram.lower()

        for doc_type in doc_types:
            if doc_type_counts[doc_type] == 0:  # Skip if no documents of this type exist
                phi_scores[cluster][doc_type] = 0
                continue

            # Construct a 2x2 contingency table for the bigram and document type
            a = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype == doc_type and bigram_lower in block.lower())  # Bigram present, doc is of type
            b = doc_type_counts[doc_type] - a  # Bigram absent, doc is of type
            c = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype != doc_type and bigram_lower in block.lower())  # Bigram present, doc not of type
            d = (total_docs - doc_type_counts[doc_type]) - c  # Bigram absent, doc not of type

            # Apply Laplace smoothing (alpha=0.5) to avoid division by zero
            smoothing = 0.5
            table = np.array([[a + smoothing, b + smoothing],
                              [c + smoothing, d + smoothing]])

            # Compute chi-squared statistic and Phi coefficient
            chi2, _, _, _ = chi2_contingency(table, correction=False)
            phi = np.sqrt(chi2 / (total_docs + 4 * smoothing)) if total_docs > 0 else 0
            phi_scores[cluster][doc_type] += phi

    return phi_scores

def plot_clusters(bigrams, labels, n_clusters, name, ax, semantic_labels, axis_limits):
    """
    Plot t-SNE clusters for a model's bigrams with semantic labels.

    Args:
        bigrams (list): List of bigrams to plot.
        labels (np.ndarray): Cluster labels for the bigrams.
        n_clusters (int): Number of clusters.
        name (str): Model name for the plot title.
        ax (matplotlib.axes.Axes): Axis object to plot on.
        semantic_labels (dict): Mapping of cluster IDs to semantic labels.
        axis_limits (tuple): Tuple of (x_limits, y_limits) for uniform axis scaling.
    """
    # Handle empty or insufficient data
    if not bigrams:
        ax.text(0.5, 0.5, 'Data Unavailable', ha='center', va='center', fontsize=12, color='red')
        ax.set_title(name)
        ax.set_xticks([])
        ax.set_yticks([])
        return

    # Vectorize bigrams into a sparse matrix of token counts
    vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
    X = vectorizer.fit_transform(bigrams).toarray()

    # Apply t-SNE for 2D visualization, dynamically tuning perplexity
    perplexity = min(30, max(5, len(bigrams)-1))
    tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)

    # Define cluster labels and colors for the 8-cluster case
    label_order = ['Math Basics', 'Math Advanced', 'Code Syntax', 'Code Structure',
                   'Literary Prose', 'Literary Themes', 'Dynamics Basics', 'Dynamics Advanced']
    colors = ['#1f77b4', '#4a90e2', '#ff7f0e', '#ffaa5e', '#2ca02c', '#6abe6a', '#d62728', '#ff6666']

    # Map clusters to labels and sort by predefined order
    cluster_mapping = []
    for i in range(n_clusters):
        cluster_label = semantic_labels.get(f'Cluster {i+1}', f'Cluster {i+1}')
        if cluster_label in label_order:
            cluster_mapping.append((i, cluster_label, label_order.index(cluster_label)))

    cluster_mapping.sort(key=lambda x: x[2])

    # Remove existing legend if present
    if ax.get_legend():
        ax.get_legend().remove()

    # Plot each cluster with its corresponding color and label
    handles = []
    for i, cluster_label, color_idx in cluster_mapping:
        idx = np.where(labels == i)[0]
        scatter = ax.scatter(tsne_coords[idx, 0], tsne_coords[idx, 1], c=[colors[color_idx]],
                             alpha=0.8, s=20)
        handles.append(scatter)

    # Add legend with cluster labels
    ax.legend(handles, [label for _, label, _ in cluster_mapping], loc='best')

    # Set plot title and axis labels
    ax.set_title(name)
    ax.set_xlabel(r't-SNE $p_1$')
    ax.set_ylabel(r't-SNE $p_2$')

    # Apply uniform axis limits if provided
    if axis_limits:
        ax.set_xlim(axis_limits[0])
        ax.set_ylim(axis_limits[1])

def assign_semantic_labels(phi_scores, n_clusters):
    """
    Assign semantic labels to clusters based on Phi coefficient scores.

    Args:
        phi_scores (dict): Phi scores mapping clusters to semantic category scores.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Mapping of cluster IDs (e.g., 'Cluster 1') to semantic labels.
    """
    labels = {}
    if n_clusters == 8:
        # Collect scores for each semantic category across clusters
        math_scores = []
        code_scores = []
        literary_scores = []
        dynamics_scores = []

        for cluster in range(n_clusters):
            scores = phi_scores[cluster]
            math_scores.append(scores['math'])
            code_scores.append(scores['code'])
            literary_scores.append(scores['literary'])
            dynamics_scores.append(scores['dynamics'])

        # Sort clusters by their association with each category
        math_pairs = sorted([(score, idx) for idx, score in enumerate(math_scores)], reverse=True)
        code_pairs = sorted([(score, idx) for idx, score in enumerate(code_scores)], reverse=True)
        literary_pairs = sorted([(score, idx) for idx, score in enumerate(literary_scores)], reverse=True)
        dynamics_pairs = sorted([(score, idx) for idx, score in enumerate(dynamics_scores)], reverse=True)

        # Assign labels to clusters, ensuring no cluster is assigned multiple labels
        used_clusters = set()
        assigned_labels = [
            ('Math Basics', math_pairs[0][1]), ('Math Advanced', math_pairs[1][1]),
            ('Code Syntax', code_pairs[0][1]), ('Code Structure', code_pairs[1][1]),
            ('Literary Prose', literary_pairs[0][1]), ('Literary Themes', literary_pairs[1][1]),
            ('Dynamics Basics', dynamics_pairs[0][1]), ('Dynamics Advanced', dynamics_pairs[1][1])
        ]

        for label, cluster_idx in assigned_labels:
            if cluster_idx not in used_clusters:
                labels[f'Cluster {cluster_idx+1}'] = label
                used_clusters.add(cluster_idx)
            else:
                # If a cluster is already used, find the next highest-scoring unused cluster
                for _, next_cluster in math_pairs + code_pairs + literary_pairs + dynamics_pairs:
                    if next_cluster not in used_clusters:
                        labels[f'Cluster {next_cluster+1}'] = label
                        used_clusters.add(next_cluster)
                        break
    return labels

def get_axis_limits(all_tsne_coords):
    """
    Compute uniform axis limits for t-SNE plots with padding.

    Args:
        all_tsne_coords (list): List of t-SNE coordinates for all models.

    Returns:
        tuple: Tuple of (x_limits, y_limits) for uniform scaling.
    """
    x_min, x_max = np.inf, -np.inf
    y_min, y_max = np.inf, -np.inf
    for coords in all_tsne_coords:
        if coords.size > 0:
            x_min = min(x_min, coords[:, 0].min())
            x_max = max(x_max, coords[:, 0].max())
            y_min = min(y_min, coords[:, 1].min())
            y_max = max(y_max, coords[:, 1].max())
    x_pad = 0.1 * (x_max - x_min)  # 10% padding on x-axis
    y_pad = 0.1 * (y_max - y_min)  # 10% padding on y-axis
    return (x_min - x_pad, x_max + x_pad), (y_min - y_pad, y_max + y_pad)

# Main execution logic for clustering and visualization
if __name__ == "__main__":
    # Define the list of models to compare
    models = ['AstroSage', 'AstroLlama2', 'Claude 3.7 Sonnet', 'Grok 3 Mini (high)', 'Llama 3.1 8B', 'o4-mini']
    n_clusters = 8  # Number of clusters for spectral clustering

    # Set up a 2x3 subplot grid for the six models
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    # First pass: Compute t-SNE coordinates to determine uniform axis limits
    all_tsne_coords = []
    for i, name in enumerate(models):
        unique_bigrams = []
        document_blocks = []

        # Load and parse model output file
        filename = f"{name}.txt"
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                content = file.read()
                # Extract reply blocks using regex patterns
                document_blocks = re.findall(r'reply\s*:\s*(.*?)(?=\nuser_answer_str:|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    document_blocks = re.findall(r'The answer should either be numeric or symbolic \(not words\)\s*.*?\n(.*?)(?=\nMESSAGE|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    print(f"Warning: No reply blocks found in {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

                # Tokenize and generate bigrams
                text = "\n".join(document_blocks)
                tokens = re.findall(r'\b\w+\b', text.lower())
                if not tokens:
                    print(f"Warning: No tokens extracted from {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

                bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
                unique_bigrams = list(Counter(bigrams).keys())
                if not unique_bigrams:
                    print(f"Warning: No unique bigrams generated from {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

        except FileNotFoundError:
            print(f"Warning: {filename} not found.")
            plot_clusters([], [], n_clusters, name, axes[i], {}, None)
            continue

        # Ensure enough bigrams for clustering
        if len(unique_bigrams) < n_clusters:
            print(f"Warning: Not enough unique bigrams ({len(unique_bigrams)}) in {name} to form {n_clusters} clusters.")
            plot_clusters([], [], n_clusters, name, axes[i], {}, None)
            continue

        # Vectorize bigrams and apply spectral clustering
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)

        # Calculate Phi coefficients and assign semantic labels
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Compute t-SNE coordinates for axis limit calculation
        perplexity = min(30, max(5, len(unique_bigrams)-1))
        tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)
        all_tsne_coords.append(tsne_coords)

        # Print semantic labels for verification
        print(f"\n{name} ({n_clusters} clusters) Semantic Labels:")
        for cluster, meaning in semantic_labels.items():
            print(f"{cluster}: {meaning}")

    # Compute uniform axis limits across all models
    axis_limits = get_axis_limits(all_tsne_coords) if all_tsne_coords else None

    # Second pass: Plot the clusters with uniform axis limits
    for i, name in enumerate(models):
        unique_bigrams = []
        document_blocks = []

        filename = f"{name}.txt"
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                content = file.read()
                document_blocks = re.findall(r'reply\s*:\s*(.*?)(?=\nuser_answer_str:|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    document_blocks = re.findall(r'The answer should either be numeric or symbolic \(not words\)\s*.*?\n(.*?)(?=\nMESSAGE|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
                text = "\n".join(document_blocks)
                tokens = re.findall(r'\b\w+\b', text.lower())
                if not tokens:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
                bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
                unique_bigrams = list(Counter(bigrams).keys())
                if not unique_bigrams:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
        except FileNotFoundError:
            plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
            continue

        if len(unique_bigrams) < n_clusters:
            plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
            continue

        # Vectorize, cluster, and label for plotting
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Plot the t-SNE clusters
        plot_clusters(unique_bigrams, labels, n_clusters, name, axes[i], semantic_labels, axis_limits)

    # Add a super title and adjust layout for the final plot
    plt.suptitle(f"t-SNE Clustering Comparison ({n_clusters} Clusters)", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save the plot as an SVG file for high-quality rendering in the paper
    plt.savefig(f"comparison_models_{n_clusters}_clusters.svg", bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
# Semantic Clustering Analysis for Language Model Outputs (4 Clusters)
# Date: May 15, 2025
# Description: This script performs a semantic analysis of language model outputs using bigram extraction,
# spectral clustering, t-SNE visualization, and Phi coefficient-based semantic labeling for 4 clusters.
# It compares six models (AstroSage, AstroLlama2, Claude 3.7 Sonnet, Grok 3 Mini (high), Llama 3.1 8B, o4-mini)
# against the PSA OpenBench Gamma dataset, as described in the accompanying paper.

import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import SpectralClustering
from collections import Counter
from scipy.stats import chi2_contingency
import warnings

# Configure matplotlib to use serif font for better readability in plots
plt.rcParams['font.family'] = 'serif'

def calculate_phi_coefficient(bigrams, labels, document_blocks, n_clusters):
    """
    Calculate Phi coefficients to quantify the association between bigrams and semantic categories.

    Args:
        bigrams (list): List of bigrams extracted from model outputs.
        labels (np.ndarray): Cluster labels assigned to each bigram.
        document_blocks (list): List of text blocks (replies) from model outputs.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Phi scores for each cluster, mapping clusters to category scores (e.g., math, code).
    """
    # Initialize a dictionary to store Phi scores for each cluster and semantic category
    phi_scores = {i: Counter() for i in range(n_clusters)}
    doc_types = ['math', 'code', 'literary', 'dynamics']

    # Classify each document block into a semantic category using regex patterns
    doc_classifications = []
    for block in document_blocks:
        block_lower = block.lower()
        if re.search(r'[0-9]+(\.[0-9]+)?|[μΔ√πθ]', block_lower):  # Math: numbers and symbols
            doc_classifications.append('math')
        elif re.search(r'(def|class|if|for|while|int|str|return|[a-z_][a-z0-9_]*(\.[a-z0-9_]+)?)', block_lower):  # Code: keywords and identifiers
            doc_classifications.append('code')
        elif re.search(r'(process|change|interaction|system|flow|evolve|adapt|transition|state|time)', block_lower):  # Dynamics: processes and temporal terms
            doc_classifications.append('dynamics')
        else:  # Literary: narrative or descriptive content
            doc_classifications.append('literary')

    total_docs = len(document_blocks)
    doc_type_counts = Counter(doc_classifications)

    # Compute Phi coefficient for each bigram and semantic category
    for i, bigram in enumerate(bigrams):
        cluster = labels[i]
        bigram_lower = bigram.lower()

        for doc_type in doc_types:
            if doc_type_counts[doc_type] == 0:  # Skip if no documents of this type exist
                phi_scores[cluster][doc_type] = 0
                continue

            # Construct a 2x2 contingency table for the bigram and document type
            a = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype == doc_type and bigram_lower in block.lower())  # Bigram present, doc is of type
            b = doc_type_counts[doc_type] - a  # Bigram absent, doc is of type
            c = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype != doc_type and bigram_lower in block.lower())  # Bigram present, doc not of type
            d = (total_docs - doc_type_counts[doc_type]) - c  # Bigram absent, doc not of type

            # Apply Laplace smoothing (alpha=0.5) to avoid division by zero
            smoothing = 0.5
            table = np.array([[a + smoothing, b + smoothing],
                              [c + smoothing, d + smoothing]])

            # Compute chi-squared statistic and Phi coefficient
            chi2, _, _, _ = chi2_contingency(table, correction=False)
            phi = np.sqrt(chi2 / (total_docs + 4 * smoothing)) if total_docs > 0 else 0
            phi_scores[cluster][doc_type] += phi

    return phi_scores

def plot_clusters(bigrams, labels, n_clusters, name, ax, semantic_labels, axis_limits):
    """
    Plot t-SNE clusters for a model's bigrams with semantic labels.

    Args:
        bigrams (list): List of bigrams to plot.
        labels (np.ndarray): Cluster labels for the bigrams.
        n_clusters (int): Number of clusters.
        name (str): Model name for the plot title.
        ax (matplotlib.axes.Axes): Axis object to plot on.
        semantic_labels (dict): Mapping of cluster IDs to semantic labels.
        axis_limits (tuple): Tuple of (x_limits, y_limits) for uniform axis scaling.
    """
    # Handle empty or insufficient data
    if not bigrams:
        ax.text(0.5, 0.5, 'Data Unavailable', ha='center', va='center', fontsize=12, color='red')
        ax.set_title(name)
        ax.set_xticks([])
        ax.set_yticks([])
        return

    # Vectorize bigrams into a sparse matrix of token counts
    vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
    X = vectorizer.fit_transform(bigrams).toarray()

    # Apply t-SNE for 2D visualization, dynamically tuning perplexity
    perplexity = min(30, max(5, len(bigrams)-1))
    tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)

    # Define cluster labels and colors for the 4-cluster case
    label_order = ['Math', 'Code', 'Literary', 'Dynamics']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    # Map clusters to labels and sort by predefined order
    cluster_mapping = []
    for i in range(n_clusters):
        cluster_label = semantic_labels.get(f'Cluster {i+1}', f'Cluster {i+1}')
        if cluster_label in label_order:
            cluster_mapping.append((i, cluster_label, label_order.index(cluster_label)))

    cluster_mapping.sort(key=lambda x: x[2])

    # Remove existing legend if present
    if ax.get_legend():
        ax.get_legend().remove()

    # Plot each cluster with its corresponding color and label
    handles = []
    for i, cluster_label, color_idx in cluster_mapping:
        idx = np.where(labels == i)[0]
        scatter = ax.scatter(tsne_coords[idx, 0], tsne_coords[idx, 1], c=[colors[color_idx]],
                             alpha=0.8, s=20)
        handles.append(scatter)

    # Add legend with cluster labels
    ax.legend(handles, [label for _, label, _ in cluster_mapping], loc='best')

    # Set plot title and axis labels
    ax.set_title(name)
    ax.set_xlabel(r't-SNE $p_1$')
    ax.set_ylabel(r't-SNE $p_2$')

    # Apply uniform axis limits if provided
    if axis_limits:
        ax.set_xlim(axis_limits[0])
        ax.set_ylim(axis_limits[1])

def assign_semantic_labels(phi_scores, n_clusters):
    """
    Assign semantic labels to clusters based on Phi coefficient scores for 4-cluster analysis.

    Args:
        phi_scores (dict): Phi scores mapping clusters to semantic category scores.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Mapping of cluster IDs (e.g., 'Cluster 1') to semantic labels.
    """
    labels = {}
    if n_clusters == 4:
        # Determine the dominant semantic category for each cluster
        sorted_scores = {k: max(v.items(), key=lambda x: x[1])[0] for k, v in phi_scores.items()}
        assigned_types = ['Math', 'Code', 'Literary', 'Dynamics']
        used_types = set()
        for cluster in range(n_clusters):
            doc_type = sorted_scores[cluster]
            # Assign labels while ensuring uniqueness
            if doc_type == 'math' and 'Math' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Math'
                used_types.add('Math')
            elif doc_type == 'code' and 'Code' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Code'
                used_types.add('Code')
            elif doc_type == 'literary' and 'Literary' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Literary'
                used_types.add('Literary')
            elif doc_type == 'dynamics' and 'Dynamics' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Dynamics'
                used_types.add('Dynamics')
            else:
                # If a category is already used, assign the next available label
                for t in assigned_types:
                    if t not in used_types:
                        labels[f'Cluster {cluster+1}'] = t
                        used_types.add(t)
                        break
    return labels

def get_axis_limits(all_tsne_coords):
    """
    Compute uniform axis limits for t-SNE plots with padding.

    Args:
        all_tsne_coords (list): List of t-SNE coordinates for all models.

    Returns:
        tuple: Tuple of (x_limits, y_limits) for uniform scaling.
    """
    x_min, x_max = np.inf, -np.inf
    y_min, y_max = np.inf, -np.inf
    for coords in all_tsne_coords:
        if coords.size > 0:
            x_min = min(x_min, coords[:, 0].min())
            x_max = max(x_max, coords[:, 0].max())
            y_min = min(y_min, coords[:, 1].min())
            y_max = max(y_max, coords[:, 1].max())
    x_pad = 0.1 * (x_max - x_min)  # 10% padding on x-axis
    y_pad = 0.1 * (y_max - y_min)  # 10% padding on y-axis
    return (x_min - x_pad, x_max + x_pad), (y_min - y_pad, y_max + y_pad)

# Main execution logic for clustering and visualization
if __name__ == "__main__":
    # Define the list of models to compare
    models = ['AstroSage', 'AstroLlama2', 'Claude 3.7 Sonnet', 'Grok 3 Mini (high)', 'Llama 3.1 8B', 'o4-mini']
    n_clusters = 4  # Number of clusters for spectral clustering

    # Set up a 2x3 subplot grid for the six models
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    # First pass: Compute t-SNE coordinates to determine uniform axis limits
    all_tsne_coords = []
    for i, name in enumerate(models):
        unique_bigrams = []
        document_blocks = []

        # Load and parse model output file
        filename = f"{name}.txt"
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                content = file.read()
                # Extract reply blocks using regex patterns
                document_blocks = re.findall(r'reply\s*:\s*(.*?)(?=\nuser_answer_str:|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    document_blocks = re.findall(r'The answer should either be numeric or symbolic \(not words\)\s*.*?\n(.*?)(?=\nMESSAGE|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    print(f"Warning: No reply blocks found in {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

                # Tokenize and generate bigrams
                text = "\n".join(document_blocks)
                tokens = re.findall(r'\b\w+\b', text.lower())
                if not tokens:
                    print(f"Warning: No tokens extracted from {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

                bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
                unique_bigrams = list(Counter(bigrams).keys())
                if not unique_bigrams:
                    print(f"Warning: No unique bigrams generated from {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

        except FileNotFoundError:
            print(f"Warning: {filename} not found.")
            plot_clusters([], [], n_clusters, name, axes[i], {}, None)
            continue

        # Ensure enough bigrams for clustering
        if len(unique_bigrams) < n_clusters:
            print(f"Warning: Not enough unique bigrams ({len(unique_bigrams)}) in {name} to form {n_clusters} clusters.")
            plot_clusters([], [], n_clusters, name, axes[i], {}, None)
            continue

        # Vectorize bigrams and apply spectral clustering
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)

        # Calculate Phi coefficients and assign semantic labels
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Compute t-SNE coordinates for axis limit calculation
        perplexity = min(30, max(5, len(unique_bigrams)-1))
        tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)
        all_tsne_coords.append(tsne_coords)

        # Print semantic labels for verification
        print(f"\n{name} ({n_clusters} clusters) Semantic Labels:")
        for cluster, meaning in semantic_labels.items():
            print(f"{cluster}: {meaning}")

    # Compute uniform axis limits across all models
    axis_limits = get_axis_limits(all_tsne_coords) if all_tsne_coords else None

    # Second pass: Plot the clusters with uniform axis limits
    for i, name in enumerate(models):
        unique_bigrams = []
        document_blocks = []

        filename = f"{name}.txt"
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                content = file.read()
                document_blocks = re.findall(r'reply\s*:\s*(.*?)(?=\nuser_answer_str:|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    document_blocks = re.findall(r'The answer should either be numeric or symbolic \(not words\)\s*.*?\n(.*?)(?=\nMESSAGE|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
                text = "\n".join(document_blocks)
                tokens = re.findall(r'\b\w+\b', text.lower())
                if not tokens:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
                bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
                unique_bigrams = list(Counter(bigrams).keys())
                if not unique_bigrams:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
        except FileNotFoundError:
            plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
            continue

        if len(unique_bigrams) < n_clusters:
            plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
            continue

        # Vectorize, cluster, and label for plotting
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Plot the t-SNE clusters
        plot_clusters(unique_bigrams, labels, n_clusters, name, axes[i], semantic_labels, axis_limits)

    # Add a super title and adjust layout for the final plot
    plt.suptitle(f"t-SNE Clustering Comparison ({n_clusters} Clusters)", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save the plot as an SVG file for high-quality rendering in the paper
    plt.savefig(f"comparison_models_{n_clusters}_clusters.svg", bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
# Semantic Clustering Analysis for Language Model Outputs (2 Clusters)
# Date: May 15, 2025
# Description: This script performs a semantic analysis of language model outputs using bigram extraction,
# spectral clustering, t-SNE visualization, and Phi coefficient-based semantic labeling for 2 clusters.
# It compares six models (AstroSage, AstroLlama2, Claude 3.7 Sonnet, Grok 3 Mini (high), Llama 3.1 8B, o4-mini)
# against the PSA OpenBench Gamma dataset, as described in the accompanying paper.

import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import SpectralClustering
from collections import Counter
from scipy.stats import chi2_contingency
import warnings

# Configure matplotlib to use serif font for better readability in plots
plt.rcParams['font.family'] = 'serif'

def calculate_phi_coefficient(bigrams, labels, document_blocks, n_clusters):
    """
    Calculate Phi coefficients to quantify the association between bigrams and semantic categories.

    Args:
        bigrams (list): List of bigrams extracted from model outputs.
        labels (np.ndarray): Cluster labels assigned to each bigram.
        document_blocks (list): List of text blocks (replies) from model outputs.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Phi scores for each cluster, mapping clusters to category scores (e.g., math, code).
    """
    # Initialize a dictionary to store Phi scores for each cluster and semantic category
    phi_scores = {i: Counter() for i in range(n_clusters)}
    doc_types = ['math', 'code', 'literary', 'dynamics']

    # Classify each document block into a semantic category using regex patterns
    doc_classifications = []
    for block in document_blocks:
        block_lower = block.lower()
        if re.search(r'[0-9]+(\.[0-9]+)?|[μΔ√πθ]', block_lower):  # Math: numbers and symbols
            doc_classifications.append('math')
        elif re.search(r'(def|class|if|for|while|int|str|return|[a-z_][a-z0-9_]*(\.[a-z0-9_]+)?)', block_lower):  # Code: keywords and identifiers
            doc_classifications.append('code')
        elif re.search(r'(process|change|interaction|system|flow|evolve|adapt|transition|state|time)', block_lower):  # Dynamics: processes and temporal terms
            doc_classifications.append('dynamics')
        else:  # Literary: narrative or descriptive content
            doc_classifications.append('literary')

    total_docs = len(document_blocks)
    doc_type_counts = Counter(doc_classifications)

    # Compute Phi coefficient for each bigram and semantic category
    for i, bigram in enumerate(bigrams):
        cluster = labels[i]
        bigram_lower = bigram.lower()

        for doc_type in doc_types:
            if doc_type_counts[doc_type] == 0:  # Skip if no documents of this type exist
                phi_scores[cluster][doc_type] = 0
                continue

            # Construct a 2x2 contingency table for the bigram and document type
            a = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype == doc_type and bigram_lower in block.lower())  # Bigram present, doc is of type
            b = doc_type_counts[doc_type] - a  # Bigram absent, doc is of type
            c = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype != doc_type and bigram_lower in block.lower())  # Bigram present, doc not of type
            d = (total_docs - doc_type_counts[doc_type]) - c  # Bigram absent, doc not of type

            # Apply Laplace smoothing (alpha=0.5) to avoid division by zero
            smoothing = 0.5
            table = np.array([[a + smoothing, b + smoothing],
                              [c + smoothing, d + smoothing]])

            # Compute chi-squared statistic and Phi coefficient
            chi2, _, _, _ = chi2_contingency(table, correction=False)
            phi = np.sqrt(chi2 / (total_docs + 4 * smoothing)) if total_docs > 0 else 0
            phi_scores[cluster][doc_type] += phi

    return phi_scores

def plot_clusters(bigrams, labels, n_clusters, name, ax, semantic_labels, axis_limits):
    """
    Plot t-SNE clusters for a model's bigrams with semantic labels.

    Args:
        bigrams (list): List of bigrams to plot.
        labels (np.ndarray): Cluster labels for the bigrams.
        n_clusters (int): Number of clusters.
        name (str): Model name for the plot title.
        ax (matplotlib.axes.Axes): Axis object to plot on.
        semantic_labels (dict): Mapping of cluster IDs to semantic labels.
        axis_limits (tuple): Tuple of (x_limits, y_limits) for uniform axis scaling.
    """
    # Handle empty or insufficient data
    if not bigrams:
        ax.text(0.5, 0.5, 'Data Unavailable', ha='center', va='center', fontsize=12, color='red')
        ax.set_title(name)
        ax.set_xticks([])
        ax.set_yticks([])
        return

    # Vectorize bigrams into a sparse matrix of token counts
    vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
    X = vectorizer.fit_transform(bigrams).toarray()

    # Apply t-SNE for 2D visualization, dynamically tuning perplexity
    perplexity = min(30, max(5, len(bigrams)-1))
    tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)

    # Define cluster labels and colors for the 2-cluster case (Technical, Narrative)
    label_order = ['Technical', 'Narrative']
    colors = ['#1f77b4', '#ff7f0e']

    # Map clusters to labels and sort by predefined order
    cluster_mapping = []
    for i in range(n_clusters):
        cluster_label = semantic_labels.get(f'Cluster {i+1}', f'Cluster {i+1}')
        if cluster_label in label_order:
            cluster_mapping.append((i, cluster_label, label_order.index(cluster_label)))

    cluster_mapping.sort(key=lambda x: x[2])

    # Remove existing legend if present
    if ax.get_legend():
        ax.get_legend().remove()

    # Plot each cluster with its corresponding color and label
    handles = []
    for i, cluster_label, color_idx in cluster_mapping:
        idx = np.where(labels == i)[0]
        scatter = ax.scatter(tsne_coords[idx, 0], tsne_coords[idx, 1], c=[colors[color_idx]],
                             alpha=0.8, s=20)
        handles.append(scatter)

    # Add legend with cluster labels
    ax.legend(handles, [label for _, label, _ in cluster_mapping], loc='best')

    # Set plot title and axis labels
    ax.set_title(name)
    ax.set_xlabel(r't-SNE $p_1$')
    ax.set_ylabel(r't-SNE $p_2$')

    # Apply uniform axis limits if provided
    if axis_limits:
        ax.set_xlim(axis_limits[0])
        ax.set_ylim(axis_limits[1])

def assign_semantic_labels(phi_scores, n_clusters):
    """
    Assign semantic labels to clusters based on Phi coefficient scores for 2-cluster analysis.

    Args:
        phi_scores (dict): Phi scores mapping clusters to semantic category scores.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Mapping of cluster IDs (e.g., 'Cluster 1') to semantic labels (Technical or Narrative).
    """
    labels = {}
    if n_clusters == 2:
        # Calculate total scores for Technical (math + code) and Narrative (literary + dynamics)
        tech_scores, narr_scores = [], []
        for cluster in phi_scores:
            scores = phi_scores[cluster]
            tech_total = scores['math'] + scores['code']
            narr_total = scores['literary'] + scores['dynamics']
            tech_scores.append(tech_total)
            narr_scores.append(narr_total)

        # Assign labels based on which category has the higher total score
        labels['Cluster 1'] = 'Technical' if tech_scores[0] > narr_scores[0] else 'Narrative'
        labels['Cluster 2'] = 'Narrative' if tech_scores[0] > narr_scores[0] else 'Technical'
    return labels

def get_axis_limits(all_tsne_coords):
    """
    Compute uniform axis limits for t-SNE plots with padding.

    Args:
        all_tsne_coords (list): List of t-SNE coordinates for all models.

    Returns:
        tuple: Tuple of (x_limits, y_limits) for uniform scaling.
    """
    x_min, x_max = np.inf, -np.inf
    y_min, y_max = np.inf, -np.inf
    for coords in all_tsne_coords:
        if coords.size > 0:
            x_min = min(x_min, coords[:, 0].min())
            x_max = max(x_max, coords[:, 0].max())
            y_min = min(y_min, coords[:, 1].min())
            y_max = max(y_max, coords[:, 1].max())
    x_pad = 0.1 * (x_max - x_min)  # 10% padding on x-axis
    y_pad = 0.1 * (y_max - y_min)  # 10% padding on y-axis
    return (x_min - x_pad, x_max + x_pad), (y_min - y_pad, y_max + y_pad)

# Main execution logic for clustering and visualization
if __name__ == "__main__":
    # Define the list of models to compare
    models = ['AstroSage', 'AstroLlama2', 'Claude 3.7 Sonnet', 'Grok 3 Mini (high)', 'Llama 3.1 8B', 'o4-mini']
    n_clusters = 2  # Number of clusters for spectral clustering

    # Set up a 2x3 subplot grid for the six models
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    # First pass: Compute t-SNE coordinates to determine uniform axis limits
    all_tsne_coords = []
    for i, name in enumerate(models):
        unique_bigrams = []
        document_blocks = []

        # Load and parse model output file
        filename = f"{name}.txt"
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                content = file.read()
                # Extract reply blocks using regex patterns
                document_blocks = re.findall(r'reply\s*:\s*(.*?)(?=\nuser_answer_str:|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    document_blocks = re.findall(r'The answer should either be numeric or symbolic \(not words\)\s*.*?\n(.*?)(?=\nMESSAGE|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    print(f"Warning: No reply blocks found in {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

                # Tokenize and generate bigrams
                text = "\n".join(document_blocks)
                tokens = re.findall(r'\b\w+\b', text.lower())
                if not tokens:
                    print(f"Warning: No tokens extracted from {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

                bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
                unique_bigrams = list(Counter(bigrams).keys())
                if not unique_bigrams:
                    print(f"Warning: No unique bigrams generated from {filename}.")
                    plot_clusters([], [], n_clusters, name, axes[i], {}, None)
                    continue

        except FileNotFoundError:
            print(f"Warning: {filename} not found.")
            plot_clusters([], [], n_clusters, name, axes[i], {}, None)
            continue

        # Ensure enough bigrams for clustering
        if len(unique_bigrams) < n_clusters:
            print(f"Warning: Not enough unique bigrams ({len(unique_bigrams)}) in {name} to form {n_clusters} clusters.")
            plot_clusters([], [], n_clusters, name, axes[i], {}, None)
            continue

        # Vectorize bigrams and apply spectral clustering
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)

        # Calculate Phi coefficients and assign semantic labels
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Compute t-SNE coordinates for axis limit calculation
        perplexity = min(30, max(5, len(unique_bigrams)-1))
        tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)
        all_tsne_coords.append(tsne_coords)

        # Print semantic labels for verification
        print(f"\n{name} ({n_clusters} clusters) Semantic Labels:")
        for cluster, meaning in semantic_labels.items():
            print(f"{cluster}: {meaning}")

    # Compute uniform axis limits across all models
    axis_limits = get_axis_limits(all_tsne_coords) if all_tsne_coords else None

    # Second pass: Plot the clusters with uniform axis limits
    for i, name in enumerate(models):
        unique_bigrams = []
        document_blocks = []

        filename = f"{name}.txt"
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                content = file.read()
                document_blocks = re.findall(r'reply\s*:\s*(.*?)(?=\nuser_answer_str:|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    document_blocks = re.findall(r'The answer should either be numeric or symbolic \(not words\)\s*.*?\n(.*?)(?=\nMESSAGE|$)', content, re.DOTALL | re.IGNORECASE)
                if not document_blocks:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
                text = "\n".join(document_blocks)
                tokens = re.findall(r'\b\w+\b', text.lower())
                if not tokens:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
                bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
                unique_bigrams = list(Counter(bigrams).keys())
                if not unique_bigrams:
                    plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
                    continue
        except FileNotFoundError:
            plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
            continue

        if len(unique_bigrams) < n_clusters:
            plot_clusters([], [], n_clusters, name, axes[i], {}, axis_limits)
            continue

        # Vectorize, cluster, and label for plotting
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Plot the t-SNE clusters
        plot_clusters(unique_bigrams, labels, n_clusters, name, axes[i], semantic_labels, axis_limits)

    # Add a super title and adjust layout for the final plot
    plt.suptitle(f"t-SNE Clustering Comparison ({n_clusters} Clusters)", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save the plot as an SVG file for high-quality rendering in the paper
    plt.savefig(f"comparison_models_{n_clusters}_clusters.svg", bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
# Reference Clustering Analysis for PSA OpenBench Gamma Dataset
# Date: May 15, 2025
# Description: This script performs a semantic analysis of the PSA OpenBench Gamma dataset using bigram extraction,
# spectral clustering, t-SNE visualization, and Phi coefficient-based semantic labeling for 2, 4, and 8 clusters.
# The analysis serves as a reference for comparing language model outputs, as described in the accompanying paper.

import re
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import SpectralClustering
from collections import Counter
from scipy.stats import chi2_contingency
import warnings

# Configure matplotlib to use serif font for better readability in plots
plt.rcParams['font.family'] = 'serif'

def calculate_phi_coefficient(bigrams, labels, document_blocks, n_clusters):
    """
    Calculate Phi coefficients to quantify the association between bigrams and semantic categories.

    Args:
        bigrams (list): List of bigrams extracted from reference solutions.
        labels (np.ndarray): Cluster labels assigned to each bigram.
        document_blocks (list): List of solution text blocks from the PSA OpenBench Gamma dataset.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Phi scores for each cluster, mapping clusters to category scores (e.g., math, code).
    """
    # Initialize a dictionary to store Phi scores for each cluster and semantic category
    phi_scores = {i: Counter() for i in range(n_clusters)}
    doc_types = ['math', 'code', 'literary', 'dynamics']

    # Classify each document block into a semantic category using regex patterns
    doc_classifications = []
    for block in document_blocks:
        block_lower = block.lower()
        if re.search(r'[0-9]+(\.[0-9]+)?|[μΔ√πθ]', block_lower):  # Math: numbers and symbols
            doc_classifications.append('math')
        elif re.search(r'(def|class|if|for|while|int|str|return|[a-z_][a-z0-9_]*(\.[a-z0-9_]+)?)', block_lower):  # Code: keywords and identifiers
            doc_classifications.append('code')
        elif re.search(r'(process|change|interaction|system|flow|evolve|adapt|transition|state|time)', block_lower):  # Dynamics: processes and temporal terms
            doc_classifications.append('dynamics')
        else:  # Literary: narrative or descriptive content
            doc_classifications.append('literary')

    total_docs = len(document_blocks)
    doc_type_counts = Counter(doc_classifications)

    # Compute Phi coefficient for each bigram and semantic category
    for i, bigram in enumerate(bigrams):
        cluster = labels[i]
        bigram_lower = bigram.lower()

        for doc_type in doc_types:
            if doc_type_counts[doc_type] == 0:  # Skip if no documents of this type exist
                phi_scores[cluster][doc_type] = 0
                continue

            # Construct a 2x2 contingency table for the bigram and document type
            a = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype == doc_type and bigram_lower in block.lower())  # Bigram present, doc is of type
            b = doc_type_counts[doc_type] - a  # Bigram absent, doc is of type
            c = sum(1 for block, dtype in zip(document_blocks, doc_classifications)
                    if dtype != doc_type and bigram_lower in block.lower())  # Bigram present, doc not of type
            d = (total_docs - doc_type_counts[doc_type]) - c  # Bigram absent, doc not of type

            # Apply Laplace smoothing (alpha=0.5) to avoid division by zero
            smoothing = 0.5
            table = np.array([[a + smoothing, b + smoothing],
                              [c + smoothing, d + smoothing]])

            # Compute chi-squared statistic and Phi coefficient
            chi2, _, _, _ = chi2_contingency(table, correction=False)
            phi = np.sqrt(chi2 / (total_docs + 4 * smoothing)) if total_docs > 0 else 0
            phi_scores[cluster][doc_type] += phi

    return phi_scores

def plot_clusters(bigrams, labels, n_clusters, name, ax, semantic_labels, axis_limits):
    """
    Plot t-SNE clusters for reference solutions with semantic labels.

    Args:
        bigrams (list): List of bigrams to plot.
        labels (np.ndarray): Cluster labels for the bigrams.
        n_clusters (int): Number of clusters.
        name (str): Model name for the plot title (set to 'Reference' here).
        ax (matplotlib.axes.Axes): Axis object to plot on.
        semantic_labels (dict): Mapping of cluster IDs to semantic labels.
        axis_limits (tuple): Tuple of (x_limits, y_limits) for uniform axis scaling.
    """
    # Handle empty or insufficient data
    if not bigrams:
        ax.text(0.5, 0.5, 'Data Unavailable', ha='center', va='center', fontsize=12, color='red')
        ax.set_title(name)
        ax.set_xticks([])
        ax.set_yticks([])
        return

    # Vectorize bigrams into a sparse matrix of token counts
    vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
    X = vectorizer.fit_transform(bigrams).toarray()

    # Apply t-SNE for 2D visualization, dynamically tuning perplexity
    perplexity = min(30, max(5, len(bigrams)-1))
    tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)

    # Define label order and colors based on number of clusters
    if n_clusters == 2:
        label_order = ['Technical', 'Narrative']
        colors = ['#1f77b4', '#ff7f0e']
    elif n_clusters == 4:
        label_order = ['Math', 'Code', 'Literary', 'Dynamics']
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    else:  # n_clusters == 8
        label_order = ['Math Basics', 'Math Advanced', 'Code Syntax', 'Code Structure',
                       'Literary Prose', 'Literary Themes', 'Dynamics Basics', 'Dynamics Advanced']
        colors = ['#1f77b4', '#4a90e2', '#ff7f0e', '#ffaa5e', '#2ca02c', '#6abe6a', '#d62728', '#ff6666']

    # Map clusters to labels and sort by predefined order
    cluster_mapping = []
    for i in range(n_clusters):
        cluster_label = semantic_labels.get(f'Cluster {i+1}', f'Cluster {i+1}')
        if cluster_label in label_order:
            cluster_mapping.append((i, cluster_label, label_order.index(cluster_label)))

    cluster_mapping.sort(key=lambda x: x[2])

    # Clear any existing legend
    if ax.get_legend():
        ax.get_legend().remove()

    # Plot and collect handles for legend
    handles = []
    for i, cluster_label, color_idx in cluster_mapping:
        idx = np.where(labels == i)[0]
        scatter = ax.scatter(tsne_coords[idx, 0], tsne_coords[idx, 1], c=[colors[color_idx]],
                             alpha=0.8, s=20)
        handles.append(scatter)

    # Set legend with explicit handles and labels
    ax.legend(handles, [label for _, label, _ in cluster_mapping], loc='best')

    ax.set_title(f'Reference ({n_clusters} clusters)')
    ax.set_xlabel(r't-SNE $p_1$')
    ax.set_ylabel(r't-SNE $p_2$')

    # Apply shared axis limits
    if axis_limits:
        ax.set_xlim(axis_limits[0])
        ax.set_ylim(axis_limits[1])

def assign_semantic_labels(phi_scores, n_clusters):
    """
    Assign semantic labels to clusters based on Phi coefficient scores.

    Args:
        phi_scores (dict): Phi scores mapping clusters to semantic category scores.
        n_clusters (int): Number of clusters.

    Returns:
        dict: Mapping of cluster IDs (e.g., 'Cluster 1') to semantic labels.
    """
    labels = {}
    if n_clusters == 2:
        # Calculate total scores for Technical (math + code) and Narrative (literary + dynamics)
        tech_scores, narr_scores = [], []
        for cluster in phi_scores:
            scores = phi_scores[cluster]
            tech_total = scores['math'] + scores['code']
            narr_total = scores['literary'] + scores['dynamics']
            tech_scores.append(tech_total)
            narr_scores.append(narr_total)

        # Assign labels based on which category has the higher total score
        labels['Cluster 1'] = 'Technical' if tech_scores[0] > narr_scores[0] else 'Narrative'
        labels['Cluster 2'] = 'Narrative' if tech_scores[0] > narr_scores[0] else 'Technical'
    elif n_clusters == 4:
        # Determine the dominant semantic category for each cluster
        sorted_scores = {k: max(v.items(), key=lambda x: x[1])[0] for k, v in phi_scores.items()}
        assigned_types = ['Math', 'Code', 'Literary', 'Dynamics']
        used_types = set()
        for cluster in range(n_clusters):
            doc_type = sorted_scores[cluster]
            if doc_type == 'math' and 'Math' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Math'
                used_types.add('Math')
            elif doc_type == 'code' and 'Code' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Code'
                used_types.add('Code')
            elif doc_type == 'literary' and 'Literary' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Literary'
                used_types.add('Literary')
            elif doc_type == 'dynamics' and 'Dynamics' not in used_types:
                labels[f'Cluster {cluster+1}'] = 'Dynamics'
                used_types.add('Dynamics')
            else:
                # If a category is already used, assign the next available label
                for t in assigned_types:
                    if t not in used_types:
                        labels[f'Cluster {cluster+1}'] = t
                        used_types.add(t)
                        break
    elif n_clusters == 8:
        # Collect scores for each semantic category across clusters
        math_scores = []
        code_scores = []
        literary_scores = []
        dynamics_scores = []

        for cluster in range(n_clusters):
            scores = phi_scores[cluster]
            math_scores.append(scores['math'])
            code_scores.append(scores['code'])
            literary_scores.append(scores['literary'])
            dynamics_scores.append(scores['dynamics'])

        # Sort clusters by their association with each category
        math_pairs = sorted([(score, idx) for idx, score in enumerate(math_scores)], reverse=True)
        code_pairs = sorted([(score, idx) for idx, score in enumerate(code_scores)], reverse=True)
        literary_pairs = sorted([(score, idx) for idx, score in enumerate(literary_scores)], reverse=True)
        dynamics_pairs = sorted([(score, idx) for idx, score in enumerate(dynamics_scores)], reverse=True)

        # Assign labels to clusters, ensuring no cluster is assigned multiple labels
        used_clusters = set()
        assigned_labels = [
            ('Math Basics', math_pairs[0][1]), ('Math Advanced', math_pairs[1][1]),
            ('Code Syntax', code_pairs[0][1]), ('Code Structure', code_pairs[1][1]),
            ('Literary Prose', literary_pairs[0][1]), ('Literary Themes', literary_pairs[1][1]),
            ('Dynamics Basics', dynamics_pairs[0][1]), ('Dynamics Advanced', dynamics_pairs[1][1])
        ]

        for label, cluster_idx in assigned_labels:
            if cluster_idx not in used_clusters:
                labels[f'Cluster {cluster_idx+1}'] = label
                used_clusters.add(cluster_idx)
            else:
                # If a cluster is already used, find the next highest-scoring unused cluster
                for _, next_cluster in math_pairs + code_pairs + literary_pairs + dynamics_pairs:
                    if next_cluster not in used_clusters:
                        labels[f'Cluster {next_cluster+1}'] = label
                        used_clusters.add(next_cluster)
                        break

    return labels

def get_axis_limits(all_tsne_coords):
    """
    Compute uniform axis limits for t-SNE plots with padding.

    Args:
        all_tsne_coords (list): List of t-SNE coordinates for all cluster configurations.

    Returns:
        tuple: Tuple of (x_limits, y_limits) for uniform scaling.
    """
    x_min, x_max = np.inf, -np.inf
    y_min, y_max = np.inf, -np.inf
    for coords in all_tsne_coords:
        if coords.size > 0:
            x_min = min(x_min, coords[:, 0].min())
            x_max = max(x_max, coords[:, 0].max())
            y_min = min(y_min, coords[:, 1].min())
            y_max = max(y_max, coords[:, 1].max())
    x_pad = 0.1 * (x_max - x_min)  # 10% padding on x-axis
    y_pad = 0.1 * (y_max - y_min)  # 10% padding on y-axis
    return (x_min - x_pad, x_max + x_pad), (y_min - y_pad, y_max + y_pad)

# Main execution logic for reference clustering analysis
if __name__ == "__main__":
    # Define the number of clusters to analyze
    n_clusters_list = [2, 4, 8]

    # Set up a 1x3 subplot grid for the three cluster configurations
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes = axes.flatten()

    # First pass: Compute t-SNE coordinates to determine uniform axis limits
    all_tsne_coords = []
    for i, n_clusters in enumerate(n_clusters_list):
        unique_bigrams = []
        document_blocks = []

        # Load and parse the PSA OpenBench Gamma dataset from JSON file
        try:
            with open('PSA_OpenBench_Gamma.json', 'r', encoding='utf-8') as f:
                dataset = json.load(f)
            # Extract solution blocks from the nested JSON structure
            document_blocks = [question['Solution'] for entry in dataset for question in entry['Questions']]
            if not document_blocks:
                print(f"Warning: No solutions found in PSA_OpenBench_Gamma.json.")
                plot_clusters([], [], n_clusters, 'Reference', axes[i], {}, None)
                continue

        except (FileNotFoundError, json.JSONDecodeError, KeyError) as e:
            print(f"Warning: Error processing PSA_OpenBench_Gamma.json: {e}")
            plot_clusters([], [], n_clusters, 'Reference', axes[i], {}, None)
            continue

        # Tokenize and generate bigrams
        text = "\n".join(document_blocks)
        tokens = re.findall(r'\b\w+\b', text.lower())
        if not tokens:
            print(f"Warning: No tokens extracted from reference solutions.")
            plot_clusters([], [], n_clusters, 'Reference', axes[i], {}, None)
            continue

        bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
        unique_bigrams = list(Counter(bigrams).keys())
        if not unique_bigrams:
            print(f"Warning: No unique bigrams generated from reference solutions.")
            plot_clusters([], [], n_clusters, 'Reference', axes[i], {}, None)
            continue

        # Ensure enough bigrams for clustering
        if len(unique_bigrams) < n_clusters:
            print(f"Warning: Not enough unique bigrams ({len(unique_bigrams)}) in Reference to form {n_clusters} clusters.")
            plot_clusters([], [], n_clusters, 'Reference', axes[i], {}, None)
            continue

        # Vectorize bigrams and apply spectral clustering
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)

        # Calculate Phi coefficients and assign semantic labels
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Compute t-SNE coordinates for axis limit calculation
        perplexity = min(30, max(5, len(unique_bigrams)-1))
        tsne_coords = TSNE(n_components=2, perplexity=perplexity, random_state=42).fit_transform(X)
        all_tsne_coords.append(tsne_coords)

        # Print semantic labels for verification
        print(f"\nReference ({n_clusters} clusters) Semantic Labels:")
        for cluster, meaning in semantic_labels.items():
            print(f"{cluster}: {meaning}")

        # Initial plot without shared axis limits
        plot_clusters(unique_bigrams, labels, n_clusters, 'Reference', axes[i], semantic_labels, None)

    # Compute uniform axis limits across all cluster configurations
    axis_limits = get_axis_limits(all_tsne_coords) if all_tsne_coords else None

    # Second pass: Plot with uniform axis limits
    for i, n_clusters in enumerate(n_clusters_list):
        unique_bigrams = []
        document_blocks = []

        # Reload and parse the dataset
        try:
            with open('PSA_OpenBench_Gamma.json', 'r', encoding='utf-8') as f:
                dataset = json.load(f)
            document_blocks = [question['Solution'] for entry in dataset for question in entry['Questions']]
            if not document_blocks:
                continue
            text = "\n".join(document_blocks)
            tokens = re.findall(r'\b\w+\b', text.lower())
            if not tokens:
                continue
            bigrams = [' '.join(tokens[j:j+2]) for j in range(len(tokens)-1)]
            unique_bigrams = list(Counter(bigrams).keys())
            if not unique_bigrams:
                continue

        except (FileNotFoundError, json.JSONDecodeError, KeyError):
            continue

        if len(unique_bigrams) < n_clusters:
            continue

        # Vectorize, cluster, and label for plotting
        vectorizer = CountVectorizer(max_features=2000, token_pattern=r'\b\w+\b', lowercase=True)
        X = vectorizer.fit_transform(unique_bigrams).toarray()
        labels = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42).fit_predict(X)
        phi_scores = calculate_phi_coefficient(unique_bigrams, labels, document_blocks, n_clusters)
        semantic_labels = assign_semantic_labels(phi_scores, n_clusters)

        # Plot with shared axis limits
        plot_clusters(unique_bigrams, labels, n_clusters, 'Reference', axes[i], semantic_labels, axis_limits)

    # Add a super title and adjust layout for the final plot
    plt.suptitle('Reference t-SNE Clustering (2, 4, 8 Clusters)', fontsize=16)
    plt.tight_layout()

    # Save the plot as an SVG file for high-quality rendering in the paper
    plt.savefig('reference_clusters_2_4_8.svg', bbox_inches='tight', dpi=300)
    plt.show()